# Analyse de données de films

Nous proposons de travailler sur des données décrivant des films. Les possibilités sont larges et vous êtes évalués sur vos propositions et votre méthodologie plus que sur vos résultats.

Les données de départ sont disponibles sur:
https://grouplens.org/datasets/movielens/
au format CSV. 

Nous nous intéresserons en particulier au jeu de données: **MovieLens 20M Dataset**. Dans ce jeu de données, vous disposez entre autre de:
* Idendifiant du film dans IMdb et TMdb (ça sera important ensuite)
* Catégorie(s) du film
* Titre du film
* Notes données par les internautes aux films

Afin de rendre le projet plus intéressant, nous ajoutons des données sur les acteurs et producteurs associés aux films (récupéré sur TMdb). Ces données sont disponibles sur les liens suivants:

http://webia.lip6.fr/~guigue/film_v2.pkl <br>
http://webia.lip6.fr/~guigue/act_v2.pkl <br>
http://webia.lip6.fr/~guigue/crew_v2.pkl

Ces fichiers contiennent respectivement : une nouvelle description des films (dont l'identifiant TMdb et la note moyenne donnée par les internautes, la date de sortie,...), une description des acteurs de chaque film et une description des équipes (scénariste, producteur, metteur en scène) pour chaque film.

Ces données sont des listes de taille 26908, chaque élément de la liste correspondant à un dictionnaire dont vous étudierez les clés pour récupérer les informations utiles.

**ATTENTION** Les contraintes de récupération d'informations en ligne font que la base MovieLens compte 27278 films mais les fichiers ci-dessus n'en comptent que 26908. Le plus simple est probablement d'éliminer les films de MovieLens qui ne sont pas dans cette seconde base.

## Consignes générales pour l'analyse des données

Vous devez proposer plusieurs analyses des données, qui devront à minima utiliser les
 techniques suivantes:
 
1. Mettre en forme les données pour identifier les acteurs et les catégories, les indexer
1. Traiter au moins un problème de régression supervisé (par exemple la prédiction de la note moyenne donnée à un film par les internautes).
1. Traiter au moins un problème de classification supervisé (par exemple la prédiction de la catégorie d'un film)
1. Utiliser les données catégorielles (catégories, acteurs,...) de manière discrète ET de manière coninue (*dummy coding*) dans des approches différentes
1. Proposer au moins une approche de catégorisation non supervisée (pour regrouper les acteurs par exemple)
1. Mener une campagne d'expérience permettant de comparer les performances sur un problème en fonction des valeurs d'un paramètre (et donc, in fine, trouver la meilleure valeur du paramètre)
1. Proposer quelques illustrations

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle as pkl


## Chargement des données (base MovieLens + enrichissements)

In [7]:
fname = "act_v2.pkl"
acteurs = pkl.load(open(fname, "rb"))
fname = "film_v2.pkl" 
movie_bis = pkl.load(open(fname, "rb"))
fname = "crew_v2.pkl" 
crew = pkl.load(open(fname, "rb"))

In [8]:
movies = pd.DataFrame(data=pd.read_csv("ml-20m/movies.csv", sep=","))
scores = pd.DataFrame(data=pd.read_csv("ml-20m/genome-scores.csv", sep=","))
ratings = pd.DataFrame(data=pd.read_csv("ml-20m/ratings.csv", sep=","))
links = pd.DataFrame(data=pd.read_csv("ml-20m/links.csv", sep=","))
tags = pd.DataFrame(data=pd.read_csv("ml-20m/tags.csv", sep=","))
genome_tags =pd.DataFrame(data=pd.read_csv("ml-20m/genome-tags.csv", sep=","))

In [34]:
scores.describe()

,movieId,tagId,relevance
count,1.170977e+07,1.170977e+07,1.170977e+07
mean,2.584297e+04,5.645000e+02,1.164833e-01
std,3.467615e+04,3.256254e+02,1.542463e-01
min,1.000000e+00,1.000000e+00,2.500000e-04
25%,2.926000e+03,2.827500e+02,2.425000e-02
50%,6.017000e+03,5.645000e+02,5.650000e-02
75%,4.606200e+04,8.462500e+02,1.415000e-01
max,1.311700e+05,1.128000e+03,1.000000e+00


In [9]:
import itertools
merged_actors = list(itertools.chain(*acteurs))
merged_crew =  list(itertools.chain(*crew))

In [10]:
df_actors = pd.DataFrame(merged_actors)
df_crew = pd.DataFrame(merged_crew)
df_movies = pd.DataFrame(movie_bis)

In [11]:
df_actors.drop(['profile_path'], inplace=True, axis=1)
df_crew.drop(['profile_path'], inplace=True, axis=1)
df_movies.drop(['backdrop_path'], inplace=True, axis=1)
df_movies.drop(['poster_path'], inplace=True, axis=1)

In [14]:
print(df_movies.isnull().sum())
print(df_crew.isnull().sum())
print(df_actors.isnull().sum())

adult                0
genre_ids            0
id                   0
original_language    0
original_title       0
overview             0
popularity           0
release_date         0
title                0
video                0
vote_average         0
vote_count           0
dtype: int64
credit_id     0
department    0
gender        0
id            0
job           0
name          0
dtype: int64
cast_id      0
character    0
credit_id    0
gender       0
id           0
name         0
order        0
dtype: int64


In [13]:
df_movies.dropna(inplace=True)

In [15]:
movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
movies['title'] = movies.title.str.replace("\((\d{4})\)","")

In [16]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [18]:
print(movies.isnull().sum())

movieId     0
title       0
genres      0
year       22
dtype: int64


In [23]:
print(tags.isnull().sum())

userId        0
movieId       0
tag          16
timestamp     0
dtype: int64


In [24]:
movies.dropna(inplace=True)
tags.dropna(inplace=True)

In [26]:
ratings["timestamp"] =pd.to_datetime(ratings["timestamp"], unit='s')
ratings["timestamp"] = ratings["timestamp"].dt.date

In [28]:
ratings.tail()

,userId,movieId,rating,timestamp
20000258,138493,68954,4.5,2009-11-13
20000259,138493,69526,4.5,2009-12-03
20000260,138493,69644,3.0,2009-12-07
20000261,138493,70286,5.0,2009-11-13
20000262,138493,71619,2.5,2009-10-17


In [35]:
movies.sort_values(by='movieId', inplace=True)
ratings.sort_values(by='movieId', inplace=True)
tags.sort_values(by='movieId', inplace=True)
links.sort_values(by='movieId', inplace=True)
scores.sort_values(by='movieId', inplace=True)


In [43]:
df = pd.merge(movies,ratings,on='movieId')

In [44]:
df.head()

,movieId,title,genres,year,userId,rating,timestamp
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,97809,3.0,2008-06-11
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,106140,5.0,2013-01-29
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,106138,3.0,2002-07-31
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,70354,4.5,2011-02-13
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,70355,3.5,2008-01-26


In [45]:
ratingCount = pd.DataFrame(df.groupby('title', as_index = False)['rating'].count().rename(columns={'rating' : 'ratingCount'}))

In [47]:
ratingCount.sort_values('ratingCount',ascending=False).head()

,title,ratingCount
18075,Pulp Fiction,67310
8387,Forrest Gump,66172
19948,"Shawshank Redemption, The",63366
20155,"Silence of the Lambs, The",63299
12138,Jurassic Park,59715


In [50]:
ratingMean = pd.DataFrame(df.groupby('title', as_index = False)['rating'].mean().rename(columns={'rating' : 'ratingMean'}))
ratingMean.sort_values('ratingMean',ascending=False).head(5)

,title,ratingMean
3099,"Bo Burnham: Words, Words, Words",5.0
11454,Inquire Within,5.0
25414,Year Zero: The Silent Death of Cambodia,5.0
134,1971,5.0
22308,The Encounter,5.0


In [51]:
ratings_count_mean = pd.merge(ratingCount,ratingMean, on='title')
ratings_count_mean.sort_values('ratingCount',ascending=False).head(5)

,title,ratingCount,ratingMean
18075,Pulp Fiction,67310,4.174231
8387,Forrest Gump,66172,4.029000
19948,"Shawshank Redemption, The",63366,4.446990
20155,"Silence of the Lambs, The",63299,4.177057
12138,Jurassic Park,59715,3.664741


In [55]:
genres_unique = pd.DataFrame(movies.genres.str.split('|').tolist()).stack().unique()
genres_unique = pd.DataFrame(genres_unique, columns=['genre'])

In [56]:
genres_unique.head()

,genre
0,Adventure
1,Animation
2,Children
3,Comedy
4,Fantasy


In [60]:
movies_genres = movies.join(movies.genres.str.get_dummies().astype(bool))
movies_genres.drop('genres', inplace=True, axis=1)

In [61]:
movies_genres.head()

,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995,False,False,True,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False
1,2,Jumanji,1995,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,Grumpier Old Men,1995,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
3,4,Waiting to Exhale,1995,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
4,5,Father of the Bride Part II,1995,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


## Feature engineering
A vous de créer les caractéristiques de description des données qui permettront d'améliorer les performances dans les tâches que vous aurez choisi d'aborder dans le projet.

In [71]:
# Faire un dictionnaire avec tous les acteurs (acteur => indice)
# + un dictionnaire inversé (indice => acteur)
actors = dict()
actors_inv = dict()
for lista in acteurs:
    for a in lista:
        # affecte une valeur à une clé si la clé n'est pas utilisé
        res = actors.setdefault(a['name'], len(actors))
        if res == len(actors)-1:
            actors_inv[len(actors)-1] = a['name']

# Exemple de transformation supplémentaire
# Dans combien de films de base joue Tom Hanks? (Réponse 57)
# Dans combien de comédies...

# => On voit qu'il est possible de créer facilement des nouvelles caractéristiques qui
# apporteront des informations utiles pour certaines tâches

In [62]:
actors_inv[0]

'Tom Hanks'

In [63]:
actors["Tom Hanks"]

0

In [73]:
genres = dict()
genres_inv = dict()
for g in genres_unique.itertuples():
    # affecte une valeur à une clé si la clé n'est pas utilisée
    res = genres.setdefault(g.genre, len(genres))
    if res == len(genres)-1:
        genres_inv[len(genres)-1] = g.genre

In [74]:
genres["Fantasy"]

4

In [78]:
df_actors.sort_values(by='id', inplace=True)

In [79]:
df_movies.sort_values(by='id', inplace=True)

In [80]:
df_actors.head()

,cast_id,character,credit_id,gender,id,name,order
267934,11,Himself,56757ade92514179d4002e15,2,1,George Lucas,7
281812,14,Himself,573de93b92514138d500088c,2,1,George Lucas,9
192825,9,Himself,52fe4684c3a36847f810367f,2,1,George Lucas,7
195635,15,Himself,52fe48dac3a36847f817cdcb,2,1,George Lucas,3
165906,28,Himself,52fe43cac3a36847f806fc33,2,1,George Lucas,3


In [81]:
df_movies.head()

,adult,genre_ids,id,original_language,original_title,overview,popularity,release_date,title,video,vote_average,vote_count
4352,False,"[18, 80, 35]",2,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,5.154,1988-10-21,Ariel,False,7.0,70
12909,False,"[35, 18]",3,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",5.123,1986-10-17,Shadows in Paradise,False,7.4,79
17,False,"[80, 35]",5,en,Four Rooms,It's Ted the Bellhop's first night on the job....,8.995,1995-12-09,Four Rooms,False,6.3,1089
475,False,"[28, 80, 53]",6,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",8.677,1993-10-15,Judgment Night,False,6.5,111
257,False,"[12, 28, 878]",11,en,Star Wars,Princess Leia is captured and held hostage by ...,35.555,1977-05-25,Star Wars,False,8.2,10843


## problème rencontré en essayant de faire des dictionnaires Films: {actors} et Actors:{categories} :
## Je ne comprends pas a quel genre correspond chaque genre_ids et comment trouver les films dans lesquels chaque acteur a jouer